# Benchmark OPF

This notebook aims to beckmark the optimal power flow.

In [1]:
import warnings
import logging
import ams
import numpy as np
import pandas as pd

ams.config_logger(stream_level=50)

%run ../benchmarks.py
_ = get_tool_versions()

Last run time: 2025-02-02 09:14:11
Python: 3.12.0 | packaged by conda-forge | (main, Oct  3 2023, 08:36:57) [Clang 15.0.7 ]

Tool        Version
----------  -------
ltbams      1.0.3a1
andes       1.9.3  
cvxpy       1.6.0  
gurobipy    12.0.1 
mosek       11.0.5 
piqp        0.4.2  
pandapower  2.14.7 
numba       0.60.0 


In [2]:
cases = [
    './cases/case14.m',
    './cases/case39.m',
    './cases/case89pegase.m',
    './cases/case118.m',
    './cases/npcc.m',
    './cases/wecc.m',
    './cases/case300.m',
    './cases/pglib_opf_case1354_pegase.m',
    './cases/pglib_opf_case2869_pegase.m',
    './cases/pglib_opf_case4020_goc.m',
    './cases/pglib_opf_case5658_epigrids.m',
    './cases/pglib_opf_case7336_epigrids.m',
]

In [3]:
# run a single case to ensure everything is working
_sp = ams.load(cases[0],
               setup=True, default_config=True, no_output=True)
_pre_time, sol = time_routine(_sp, routine='DCOPF',
                              solvers=['GUROBI', 'MOSEK',
                                       'PIQP', 'pandapower'],
                              ignore_dpp=True)

print(f"Test run successful.\nTime: {_pre_time} s\nSolution: {sol}")

Set parameter Username
Set parameter LicenseID to value 2617183
Academic license - for non-commercial use only - expires 2026-02-02


/Users/jinningwang/work/miniconda3/envs/amsre/lib/python3.12/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/Users/jinningwang/work/miniconda3/envs/amsre/lib/python3.12/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/Users/jinningwang/work/miniconda3/envs/amsre/lib/python3.12/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");
/Users/jinningwang/work/miniconda3/envs/amsre/lib/python3.12/site-packages/pandapower/create.py:5049: FutureWarning: The 

Test run successful.
Time: {'ams_mats': 0.0007, 'ams_parse': 0.0021, 'ams_eval': 0.0024, 'ams_final': 0.0001, 'ams_postinit': 0.0} s
Solution: {'GUROBI': {'time': 0.0129, 'obj': 7642.591781969892}, 'MOSEK': {'time': 0.0187, 'obj': 7642.591729476251}, 'PIQP': {'time': 0.0036, 'obj': 7642.591777048095}, 'pandapower': {'time': 0.0829, 'obj': 7642.591776991807}}


In [4]:
# Suppress warnings
warnings.filterwarnings('ignore')
# show only errors
logging.getLogger('pandapower').setLevel(logging.ERROR)

# Initialize data structures for storing results
n_iters = 10
n_cases = len(cases)

solvers = ['GUROBI', 'MOSEK', 'PIQP', 'pandapower']

time_data = np.zeros((n_iters, n_cases, len(cols_pre) + len(solvers)))
obj_data = np.zeros((n_cases, len(solvers)))

In [5]:
# Run the benchmark for each case and iteration
for n_case, case in enumerate(cases):
    print(f'Case: {case}')
    system = ams.load(case, setup=True, default_config=True, no_output=True)
    if system.Bus.n > 4000:
        system.Line.set(src='rate_a', attr='v',
                        idx=system.Line.idx.v, value=99999)
    for n_iter in range(n_iters):
        pre_time, sol = time_routine(system, routine='DCOPF',
                                     solvers=solvers,
                                     ignore_dpp=True)
        time_data[n_iter, n_case, :len(pre_time)] = np.array(
            [i for i in pre_time.values()])
        time_data[n_iter, n_case, len(pre_time):] = np.array(
            [sol[solver]['time'] for solver in solvers])
    obj = np.array([sol[solver]['obj'] for solver in solvers])
    obj_data[n_case, :] = obj

# Optionally, re-enable warnings if needed elsewhere in your code
warnings.filterwarnings('default')

Case: ./cases/case14.m
Case: ./cases/case39.m
Case: ./cases/case89pegase.m
Case: ./cases/case118.m
Case: ./cases/npcc.m
Case: ./cases/wecc.m
Case: ./cases/case300.m
Case: ./cases/pglib_opf_case1354_pegase.m
Case: ./cases/pglib_opf_case2869_pegase.m
Case: ./cases/pglib_opf_case4020_goc.m
Case: ./cases/pglib_opf_case5658_epigrids.m
Case: ./cases/pglib_opf_case7336_epigrids.m


In [6]:
# Process and display the results
case_names = [case.split('/')[-1].split('.')[0] for case in cases]

obj_dcopf = pd.DataFrame(obj_data, columns=solvers, index=case_names)

time_dcopf = pd.DataFrame(columns=cols_pre+solvers, index=case_names)
for case in case_names:
    time_dcopf.loc[case] = time_data[:, case_names.index(case), :].mean(axis=0)

time_dcopf.iloc[:, :] *= 1000  # Convert time to milliseconds

In [7]:
time_dcopf

,ams_mats,ams_parse,ams_eval,ams_final,ams_postinit,GUROBI,MOSEK,PIQP,pandapower
case14,0.63,1.13,1.77,0.09,0.0,10.26,5.1,3.47,12.81
case39,0.62,1.13,1.84,0.08,0.0,5.11,5.52,3.55,13.01
case89pegase,1.16,1.11,1.91,0.08,0.0,7.67,7.46,4.44,18.85
case118,1.16,1.13,4.79,0.09,0.0,7.91,9.33,4.36,25.94
npcc,1.27,1.13,4.23,0.09,0.0,9.1,10.08,4.65,62.0
wecc,1.38,1.12,2.77,0.09,0.0,8.36,8.54,4.69,37.94
case300,1.88,1.12,6.68,0.09,0.0,13.04,14.94,5.88,37.21
pglib_opf_case1354_pegase,7.81,1.15,63.69,0.2,0.0,45.86,41.62,17.01,556.91
pglib_opf_case2869_pegase,17.38,1.17,235.67,0.49,0.0,129.31,105.88,39.93,1603.68
pglib_opf_case4020_goc,22.4,1.14,117.81,0.89,0.0,402.21,187.78,169.76,2068.98


In [8]:
obj_dcopf.round(2)

,GUROBI,MOSEK,PIQP,pandapower
case14,7642.59,7642.59,7642.59,7642.59
case39,41263.94,41263.94,41263.94,41263.94
case89pegase,5733.37,5733.37,5733.37,5733.37
case118,125947.88,125947.88,125947.88,125947.88
npcc,810033.37,810033.37,810033.37,810016.06
wecc,411706.13,411706.13,411706.13,411706.13
case300,706292.32,706292.32,706292.32,706292.32
pglib_opf_case1354_pegase,1218096.86,1218096.86,1218096.86,1218096.86
pglib_opf_case2869_pegase,2386235.33,2386235.32,2386235.33,2386235.33
pglib_opf_case4020_goc,793634.43,793634.11,793634.11,793634.11


In [9]:
# export results to csv
obj_dcopf.to_csv('../results/results_obj.csv')
time_dcopf.to_csv('../results/results_time.csv')